# Keyword Spotting baseline classifier

In [1]:
import numpy as np
np.random.seed(1234)
from utils import load_dataset_keywords
import sklearn
import pickle as pk
print(f'{sklearn.__version__ = }')

sklearn.__version__ = '1.2.0'


## Data loading

In [2]:
data_dir = 'speechdataset/processed_data'

keywords = ['marvin', 'no', 'yes']

categories = ['bed', 'down', 'forward', 'house', 'nine', 'one', 'six', 'tree', 'visual', 'bird', 'eight', 
              'four', 'learn', 'no','right', 'stop', 'two', 'wow', 'cat', 'five', 'go', 'left', 'off', 
              'seven', 'up', 'yes', 'backward', 'dog', 'follow', 'happy', 'marvin', 'on', 'sheila', 'three', 'zero']

n_labels = len(keywords) + 1 # number of keywords + not a keyword
# Best preprocessed data
X_train, Y_train, X_test, Y_test = load_dataset_keywords(data_dir, keywords, categories, frames=66,
                                                                 winlen=0.025, winstep=0.02, nfilt=26) # Time x Features x Number

In [3]:
print(X_train.shape, Y_train.shape)

(66, 39, 89954) (89954,)


## PCA + SVM as a baseline

In [4]:
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import time

kept_dimensions = int(0.1 * (X_train.shape[0] * X_train.shape[1])) 
pca = PCA(kept_dimensions)
# Need Number x (Time * Features) shape for PCA
Z_train = pca.fit_transform(X_train.transpose(2, 0, 1).reshape(X_train.shape[2], X_train.shape[0] * X_train.shape[1]))
print(f'Variance explained with {kept_dimensions} dimensions: {100 * sum(pca.explained_variance_ratio_):2.2f} %')
Z_test = pca.transform(X_test.transpose(2, 0, 1).reshape(X_test.shape[2], X_test.shape[0] * X_test.shape[1]))
# SVM for multiclass One-vs-Rest
svm = LinearSVC()
baseline_classifier = OneVsRestClassifier(svm, verbose=5, n_jobs=15)
baseline_classifier.fit(Z_train, Y_train)
prediction = baseline_classifier.predict(Z_test)
# Measure performance
count = 0
for idx, label in enumerate(prediction):
    if label == Y_test[idx]:
        count += 1
acc = 100 * count / len(Y_test)
time.sleep(3)
print(f'PCA + SVM accuracy:\t {acc:2.2f} %')
print(f'Random label accuracy:\t {100 / n_labels:2.2f} %')

Variance explained with 257 dimensions: 92.63 %


[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 out of   4 | elapsed:  1.9min remaining:  1.9min
[Parallel(n_jobs=15)]: Done   4 out of   4 | elapsed:  2.7min remaining:    0.0s
[Parallel(n_jobs=15)]: Done   4 out of   4 | elapsed:  2.7min finished


PCA + SVM accuracy:	 88.89 %
Random label accuracy:	 25.00 %


In [5]:
# Save model
pk.dump(pca, open(f"pca_{kept_dimensions}dim_best_preprocess.pkl","wb"))
pk.dump(baseline_classifier, open(f"baselineClassifier_{kept_dimensions}dim_best_preprocess.pkl","wb"))

In [6]:
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import time

kept_dimensions = int(0.01 * (X_train.shape[0] * X_train.shape[1])) 
pca = PCA(kept_dimensions)
# Need Number x (Time * Features) shape for PCA
Z_train = pca.fit_transform(X_train.transpose(2, 0, 1).reshape(X_train.shape[2], X_train.shape[0] * X_train.shape[1]))
print(f'Variance explained with {kept_dimensions} dimensions: {100 * sum(pca.explained_variance_ratio_):2.2f} %')
Z_test = pca.transform(X_test.transpose(2, 0, 1).reshape(X_test.shape[2], X_test.shape[0] * X_test.shape[1]))
# SVM for multiclass One-vs-Rest
svm = LinearSVC()
baseline_classifier = OneVsRestClassifier(svm, verbose=5, n_jobs=15)
baseline_classifier.fit(Z_train, Y_train)
prediction = baseline_classifier.predict(Z_test)
# Measure performance
count = 0
for idx, label in enumerate(prediction):
    if label == Y_test[idx]:
        count += 1
acc = 100 * count / len(Y_test)
time.sleep(3)
print(f'PCA + SVM accuracy:\t {acc:2.2f} %')
print(f'Random label accuracy:\t {100 / n_labels:2.2f} %')

Variance explained with 25 dimensions: 49.34 %


[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 out of   4 | elapsed:   26.9s remaining:   26.9s
[Parallel(n_jobs=15)]: Done   4 out of   4 | elapsed:   35.3s remaining:    0.0s
[Parallel(n_jobs=15)]: Done   4 out of   4 | elapsed:   35.3s finished


PCA + SVM accuracy:	 86.35 %
Random label accuracy:	 25.00 %


In [7]:
# Save model
pk.dump(pca, open(f"pca_{kept_dimensions}dim_best_preprocess.pkl","wb"))
pk.dump(baseline_classifier, open(f"baselineClassifier_{kept_dimensions}dim_best_preprocess.pkl","wb"))

In [9]:
# Load saved model
pca_reload = pk.load(open('pca_257dim_best_preprocess.pkl','rb'))
Z_test = pca_reload.transform(X_test.transpose(2, 0, 1).reshape(X_test.shape[2], X_test.shape[0] * X_test.shape[1]))
BClassifier = pk.load(open('baselineClassifier_257dim_best_preprocess.pkl', 'rb'))
prediction = BClassifier.predict(Z_test)

# Measure performance
count = 0
for idx, label in enumerate(prediction):
    if label == Y_test[idx]:
        count += 1
acc = 100 * count / len(Y_test)
time.sleep(3)
print(f'PCA + SVM accuracy:\t {acc:2.2f} %')
print(f'Random label accuracy:\t {100 / n_labels:2.2f} %')

PCA + SVM accuracy:	 88.89 %
Random label accuracy:	 25.00 %
